In [ ]:
import json
import os

import numpy as np
import pandas as pd
from google.cloud import aiplatform, bigquery

In [ ]:
PROJECT_ID = "pongthorn"

In [ ]:
# bqclient = bigquery.Client(PROJECT_ID)

# def download_table(bq_table_uri: str):
#     # Remove bq:// prefix if present
#     prefix = "bq://"
#     if bq_table_uri.startswith(prefix):
#         bq_table_uri = bq_table_uri[len(prefix) :]

#     table = bigquery.TableReference.from_string(bq_table_uri)
#     rows = bqclient.list_rows(
#         table,
#     )
#     return rows.to_dataframe()

def calculate_mean_and_std(df):
    # Calculate mean and std for each applicable column
    mean_and_std = {}
    dtypes = list(zip(df.dtypes.index, map(str, df.dtypes)))
    # Normalize numeric columns.
    for column, dtype in dtypes:
        if dtype == "float32" or dtype == "float64":
            mean_and_std[column] = {
                "mean": df[column].mean(),
                "std": df[column].std(),
            }

    return mean_and_std



In [ ]:
# Define the BigQuery source dataset
# BQ_SOURCE = "bq://pongthorn.SMartML.TrainEval_Incident_20230316"
# dataframe = download_table(BQ_SOURCE)

root_path='../../data'    
dataframe = pd.read_csv(f"{root_path}/train_incident.csv")
print(dataframe.info())

In [ ]:
mean_and_std = calculate_mean_and_std(dataframe)
print(f"The mean and stds for each column are: {str(mean_and_std)}")

# Write to a file
MEAN_AND_STD_JSON_FILE = "incident_mean_and_std.json"

with open(MEAN_AND_STD_JSON_FILE, "w") as outfile:
    json.dump(mean_and_std, outfile)
